In [1]:
import re
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import time
import glob
from joblib import Parallel, delayed
import xarray as xr
import sys
sys.path.insert(1, '/home/b/b382634/owl/owl')
from HWMI import *

In [2]:
### SET-UP ###
var="tmax" #tmax,tmin

percent_thresh=90
duration_min=3

### TIME ###
clim_start=1981 #1981
clim_end=2010 #2010
period_start=1940
period_end=2022
if (period_end<clim_end) or (period_start>clim_start):
    raise Exception("Climatology period falls outside of target period")
years=[*range(period_start,period_end+1)]
climyears=[*range(clim_start,clim_end+1)]

nyear=len(years)
nclimyear=len(climyears)
nday=123 # days in period May to August
nrealisation=1 # ensemble members

In [3]:
# Choose location #
# LAKE COMO VALLE #
region="LakeComo_Valle"
lon = '''9°5'0"E'''
lat = '''45°49'0"N'''

deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
latP= (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
deg, minutes, seconds, direction =  re.split('[°\'"]',lon)
lonP= (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


In [4]:
data=np.zeros((nyear,nday,1))

times=[]
for iyear,year in enumerate(years):
    print (year)
    if var=="atemp2m_night_russo":
        dataset=Dataset("/work/bk1318/b382634/ERA5/daily/{0}/atemp2m_night_{0}_EUR_MJJA.nc".format(year),'r')
        tmax=dataset['atemp2m_night'][:]
    else:   
        dataset=Dataset("/work/bk1318/b382634/ERA5/daily/{0}/{0}_{1}_EUR_MJJA.nc".format(var,year),'r')
        tmax=dataset['{0}'.format(var)][:]
    time=dataset['time'][:]

    if iyear==0:   
        lons=dataset['longitude'][:]
        lats=dataset['latitude'][:]
        ind_lon=np.argwhere(lons==find_nearest(lons,lonP))[0]
        ind_lat=np.argwhere(lats==find_nearest(lats,latP))[0]
    
    times.append(time.tolist())

    for t in range(tmax.shape[0]):
        data[iyear,t]=tmax[t,ind_lat,ind_lon]
        #print (year,t,np.mean(tmask))

1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [5]:
print (data.shape)

(83, 123, 1)


In [6]:
def parallelized_HWMIs_computation(target):
    HWMIyear, HWlstyear, HWstartmembyear, HWendmembyear, ndayexedthresholdyear, DDthresholdyear, subHWarrayyear, fitsubHWarrayyear,sstMeanarrayyear, impossible_fit, perc_thresh_clim, exed_percentile, mask_duration_min, mean_clim = calc_HWMIyear(target, climyears, years, cross_valid = False, percent_thresh = percent_thresh, duration_min = duration_min, opt="polyfit")
    
    HW_arg=np.argwhere(mask_duration_min.flatten()>0)
    intensity=np.zeros(((nyear)*nday))
    for arg in HW_arg:
        arg=int(arg)
        dur=int(mask_duration_min.flatten()[arg])
        intensity[arg:arg+dur]=exed_percentile.flatten()[arg:arg+dur]

    occurrence=np.copy(intensity)
    occurrence[occurrence>0]=1
    intensity=np.ma.masked_where(intensity==0,intensity)
    occurrence=np.reshape(occurrence,(nyear,nday))
    intensity=np.reshape(intensity,(nyear,nday))
    
    return np.array(occurrence), np.array(intensity)

In [7]:
print (data.shape)
HW_occ, HW_int=parallelized_HWMIs_computation(data)

(83, 123, 1)


/tmp/ipykernel_779420/3613734751.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  arg=int(arg)


In [8]:
print (HW_occ)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
